1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
   - 토큰 수 초과로 답변을 생성하지 못할 수 있음
   - 문서가 길면 (input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [ ]:
# DOCX 파일을 로드하기 위한 로더 생성
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# tax.docx 파일을 읽을 로더 객체 생성
loader = Docx2txtLoader("./tax_with_markdown.docx")

# 문서 로드 (Document 객체 리스트 반환)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()  # .env 파일에서 환경 변수 로드

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-markdown-index"

database = PineconeVectorStore.from_documents(
    document_list,
    embedding,
    index_name=index_name,
)

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 유사도 검색 테스트
retrieved_docs = database.similarity_search(query, k=5)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
prompt = f"""
[Identity]

- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [ ]:
ai_message = llm.invoke(prompt)

ai_message